    # TP Intelligence Artificielle - Recherche Arborescente Non Informée
    # 人工智能实验 - 无信息树搜索

# **Partie 0 : Visualisation des états**
    # **第 0 部分：状态可视化**

## Voici une fonction pour visualiser les états. Nous l'utiliserons plus tard.
    ## 这是一个可视化状态的函数。我们稍后会用到它。

In [3]:
from IPython.display import display, HTML

def visualize_state(state):
    """Visualizes the given state of the Taquin using HTML."""
    """使用 HTML 可视化给定的推盘游戏状态。"""
    html = "<table>"
    for row in state:
        html += "<tr>"
        for tile in row:
            if tile == 0:
                html += "<td style='background-color: lightgray; width: 30px; height: 30px; text-align: center; font-size: 20px;'> </td>"  # Blank tile
            else:
                html += f"<td style='background-color: lightblue; width: 30px; height: 30px; text-align: center; font-size: 20px;'>{tile}</td>"
        html += "</tr>"
    html += "</table>"
    display(HTML(html))

    # **Partie 1 : Modélisation**
    # **第 1 部分：建模**

### Nous allons créer deux classes pour modeliser le taquin en espace d'états.
### 我们将创建两个类来在状态空间中为推盘游戏建模。\n

1. **Taquin** : Cette classe représente le problème du jeu du Taquin. Elle contient :
1. **Taquin**：此类代表推盘游戏问题。它包含：\n

  * Attributs:

      * initial_state : L'état initial du Taquin, représenté par une liste de listes. Chaque sous-liste représente une ligne du Taquin, et chaque élément de la sous-liste représente une tuile. La tuile vide est représentée par le chiffre 0.
      * initial_state：推盘游戏的初始状态，由列表的列表表示。每个子列表代表推盘游戏的一行，子列表的每个元素代表一个图块。空图块由数字 0 表示。\n
      * goal_state : L'état but du Taquin, représenté de la même manière que l'état initial.
      * goal_state：推盘游戏的目标状态，以与初始状态相同的方式表示。\n
      * size : La taille du Taquin (par exemple, 3 pour un Taquin 3x3, 4 pour un Taquin 4x4).
      * size：推盘游戏的大小（例如，3 代表 3x3 推盘游戏，4 代表 4x4 推盘游戏）。\n

  * Méthodes:

      * actions(state) : Cette méthode prend un état du Taquin en entrée et retourne une liste des actions possibles à partir de cet état. Les actions possibles sont "haut", "bas", "gauche" et "droite", représentant les mouvements possibles de la tuile vide.
      * actions(state)：此方法将推盘游戏的状态作为输入，并返回从该状态开始的可能操作列表。可能的操作是“上”、“下”、“左”和“右”，代表空图块的可能移动。\n
      * result(state, action) : Cette méthode prend un état et une action en entrée et retourne le nouvel état du Taquin après avoir appliqué l'action à l'état.
      * result(state, action)：此方法将状态和操作作为输入，并返回将操作应用于状态后的推盘游戏新状态。\n
      * is_goal(state) : Cette méthode prend un état en entrée et retourne True si cet état est l'état but, False sinon.
      * is_goal(state)：此方法将状态作为输入，如果此状态是目标状态，则返回 True，否则返回 False。\n
      * cost(state, action) : Cette méthode retourne le coût de l'application d'une action à un état donné. Dans le cas du Taquin, le coût est généralement constant et égal à 1 pour chaque action.
      * cost(state, action)：此方法返回将操作应用于给定状态的成本。在推盘游戏的情况下，成本通常是恒定的，每个操作的成本等于 1。\n
  
2.   **Node** :  Cette classe représentera un nœud dans l'arbre de recherche. Ces attributs sont :
2.   **Node**：此类将代表搜索树中的一个节点。这些属性是：\n

  * state : L'état représenté par ce nœud.
  * state：此节点代表的状态。\n
  * parent : Un pointeur vers le nœud parent (None pour le nœud racine).
  * parent：指向父节点的指针（根节点为 None）。\n
  * action : L'action qui a conduit à ce nœud à partir du nœud parent (None pour le nœud racine).
  * action：导致从父节点到达此节点的操作（根节点为 None）。\n
  * path_cost : Le coût total du chemin depuis le nœud racine jusqu'à ce nœud (facultatif, pour les algorithmes avec des considérations de coût).
  * path_cost：从根节点到此节点的路径总成本（可选，用于考虑成本的算法）。\n
  * depth : La profondeur de ce nœud dans l'arbre (facultatif, pour la recherche en profondeur limitée).
  * depth：此节点在树中的深度（可选，用于有限深度搜索）。\n



--------------------------------------------------------------------------------
## **1.1 Classe Taquin**
--------------------------------------------------------------------------------

In [4]:
class Taquin:
    """
    A class representing the Taquin problem.
    代表推盘游戏问题的类。
    """

    def __init__(self, initial_state, goal_state, size):
        self.initial_state = initial_state
        self.goal_state = goal_state
        self.size = size

    def actions(self, state):
        """Returns the possible actions (moves) from the given state."""
        """返回从给定状态可能的动作（移动）。"""
        # Find the position of the blank tile (0)
        # 找到空图块 (0) 的位置
        row, col = next(
            (r, c)
            for r, row in enumerate(state)
            for c, val in enumerate(row)
            if val == 0
        )

        # Define possible moves (up, down, left, right)
        # 定义可能的动作（上、下、左、右）
        possible_actions = []
        if row > 0:
            possible_actions.append("up")
        if row < self.size-1:
            possible_actions.append("down")
        if col > 0:
            possible_actions.append("left")
        if col < self.size-1:
            possible_actions.append("right")

        return possible_actions

    def result(self, state, action):
        """Returns the state that results from applying the given action."""
        """返回应用给定动作后产生的结果状态。"""
        # Create a copy of the state to avoid modifying the original
        # 创建状态的副本以避免修改原始状态
        new_state = [list(row) for row in state]

        # Find the position of the blank tile (0)
        # 找到空图块 (0) 的位置
        row, col = next(
            (r, c)
            for r, row in enumerate(state)
            for c, val in enumerate(row)
            if val == 0
        )

        # Apply the action to move the blank tile
        # 应用动作以移动空图块
        if action == "up":
            new_state[row][col], new_state[row - 1][col] = (
                new_state[row - 1][col],
                new_state[row][col],
            )
        elif action == "down":
            new_state[row][col], new_state[row + 1][col] = (
                new_state[row + 1][col],
                new_state[row][col],
            )
        elif action == "left":
            new_state[row][col], new_state[row][col - 1] = (
                new_state[row][col - 1],
                new_state[row][col],
            )
        elif action == "right":
            new_state[row][col], new_state[row][col + 1] = (
                new_state[row][col + 1],
                new_state[row][col],
            )

        return new_state

    def is_goal(self, state):
        return state == self.goal_state  # Directly compare with goal_state

    def cost(self, state, action):
        return 1  # Default cost is 1

## **Exercise 1**
## **练习 1**
1. Créez un jeu de taquin 4x4 avec un état initial et un état objectif, puis visualisez ces deux états.
1. 创建一个具有初始状态和目标状态的 4x4 推盘游戏，然后可视化这两个状态。\n
2. Identifiez les actions possibles à partir de l'état initial.
2. 确定从初始状态开始的可能操作。\n
3. Appliquez une des actions possibles et visualisez le nouvel état.
3. 应用其中一个可能的操作并可视化新状态。

In [5]:
from Taquin import Taquin
from Taquin_pr import visualize_state

initial_state = [
    [1,2,3,4],
    [5,6,7,8],
    [9,10,0,11],
    [12,13,14,15]
]

goal_state = [
    [1,2,3,4],
    [5,6,7,8],
    [9,10,11,12],
    [13,14,15,16]
]

t = Taquin(initial_state, goal_state, size=3)

# 展示初始状态
visualize_state(initial_state)

# 示例：执行一步动作并展示
actions = t.actions(initial_state)
next_state = t.result(initial_state, actions[0])
visualize_state(next_state)

1,2,3,4
5,6,7,8
9,10,,11
12,13,14,15


1,2,3,4
5,6,,8
9,10,7,11
12,13,14,15


--------------------------------------------------------------------------------
## **1.2 Classe Node**
--------------------------------------------------------------------------------

In [3]:
class Node:
    """
    A node in a search tree.
    搜索树中的一个节点。
    __init__: Initializes a node with its state, parent, action, path cost, and depth.
    __init__：使用其状态、父节点、动作、路径成本和深度初始化节点。
    __repr__: Provides a string representation of the node.
    __lt__: Defines a comparison operator for nodes based on their states.
    expand: Generates child nodes by applying all possible actions.
    child_node: Creates a single child node for a given action.
    solution: Returns the sequence of actions that led to this node.
    path: Returns the path from the root to this node as a list of nodes.
    path：返回从根节点到此节点的路径（节点列表）。
    """

    def __init__(self, state, parent=None, action=None, path_cost = 0):
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0 if parent is None else parent.depth + 1

    def expand(self, problem):
        """List the nodes reachable in one step from this node."""
        """列出从此节点一步可达的节点。"""
        return [
            self.child_node(problem, action)
            for action in problem.actions(self.state)
        ]

    def child_node(self, problem, action):
        """Create a child node by applying the given action."""
        """通过应用给定的动作创建一个子节点。"""
        next_state = problem.result(self.state, action)
        next_node = Node(
            next_state,
            parent=self,
            action=action,
            path_cost = self.path_cost + problem.cost(self.state, action),
        )
        return next_node

    def solution(self):
        """Return the sequence of actions to go from the root to this node."""
        """返回从根节点到此节点的动作序列。"""
        return [node.action for node in self.path()[1:]]

    def path(self):
        """Return a list of nodes forming the path from the root to this node."""
        """返回形成从根节点到此节点路径的节点列表。"""
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))

## **Exercise 2** 
## **练习 2**
1. Créez un jeu de taquin 3x3 avec un état initial et un état objectif.
1. 创建一个具有初始状态和目标状态的 3x3 推盘游戏。\n
2. Créez un nœud représentant l'état initial du jeu.
2. 创建一个代表游戏初始状态的节点。\n
3. Visualisez les enfants de l'état initial.
3. 可视化初始状态的子节点。\n
4. Pour chaque enfant, imprimez :
4. 对于每个子节点，打印：\n
    * L'action à effectuer pour passer de l'état initial à cet enfant.
    * 从初始状态移动到该子节点所执行的动作。\n
    * Le coût associé à cette action.
    * 与此动作关联的成本。\n
5. Répétez toutes les étapes pour un jeu de taquin 4x4.
5. 对 4x4 推盘游戏重复所有步骤。

# **Partie 2 : BFS et DFS**
    # **第 2 部分：BFS 和 DFS**


Dans cette 2ème partie nous allons coder les algorithmes de recherche arborescente que nous avons étudiés en cours.
    在这一部分中，我们将编写我们在课程中学习的树搜索算法。

## **2.1 Breadth First Search**
    ## **2.1 广度优先搜索**


### **Exercice 3** 

Créer une fonction BFS qui prend un objet problem comme entrée et exécute l'algorithme de recherche en profondeur. Votre fonction doit :
Créer une fonction BFS qui prend un objet problem comme entrée et exécute l'algorithme de recherche en profondeur. Votre fonction doit :
    创建一个 BFS 函数，该函数以 problem 对象作为输入并执行广度优先搜索算法。您的函数必须：\n

1. Retourner le nœud objectif trouvé ainsi que le nombre de nœuds explorés.
1. 返回找到的目标节点以及探索的节点数。\n
2. Retourner None si l'algorithme explore tout l'arbre sans trouver le nœud objectif.
2. 如果算法探索了整棵树但没有找到目标节点，则返回 None。\n
3. Prendre en compte un budget d'exploration pour arrêter l'algorithme si aucune solution n'est trouvée après plusieurs itérations (utile pour les grands problèmes). Le budget sera également une entrée de la fonction, avec float('inf') comme valeur par défaut.
3. 考虑探索预算，如果在多次迭代后未找到解，则停止算法（对于大型问题很有用）。预算也将是函数的输入，默认值为 float('inf')。\n

Pour la frontière et l'ensemble des nœuds déjà explorés :
对于边界和已探索节点集合：\n
* La frontière doit être une liste de nœuds (objets créés par la classe Node).
* 边界必须是节点列表（由 Node 类创建的对象）。\n
* L'ensemble des nœuds explorés peut être une liste d'états.
* 已探索节点集合可以是状态列表。\n
* Pour sélectionner un élément de la frontière à explorer, utilisez la méthode *pop*. Assurez-vous de toujours prendre le premier élément de la liste.
* 要选择要探索的边界元素，请使用 *pop* 方法。确保总是取出列表中的第一个元素。\n

Pour ajouter un élément à une liste, utilisez *liste.append(element)*.

### **Exercice 4**
### **练习 4**
1. Appliquez votre fonction BFS pour résoudre un Taquin 3x3 avec l'état initial
1. 应用您的 BFS 函数解决具有初始状态的 3x3 推盘游戏\n
   [[1, 2, 3], [4, 5, 6], [0, 7, 8]].
   Imprimez :
   打印：\n
    * Le nombre de nœuds explorés par l'algorithme.
    * 算法探索的节点数。\n
    * Le chemin pour passer de l'état initial à l'état objectif (la liste des actions effectuées).
    * 从初始状态到目标状态的路径（执行的动作列表）。\n
    * Les états successifs du chemin (utilisez la fonction visualize_state).
    * 路径的连续状态（使用 visualize_state 函数）。\n
3. Répétez la même expérience pour l'état initial [[1, 2, 3], [4, 5, 0], [6, 7, 8]].
3. 对初始状态 [[1, 2, 3], [4, 5, 0], [6, 7, 8]] 重复相同的实验。

## **2.2 Depth First Search**
    ## **2.2 深度优先搜索**

### **Exercice 5**
### **练习 5**
Créer une fonction DFS qui prend un objet problem comme entrée et exécute l'algorithme de recherche en profondeur. Votre fonction doit :
创建 DFS 函数，输入 problem 对象并执行深度优先搜索算法。您的函数必须：\n

1. Retourner le nœud objectif trouvé ainsi que le nombre de nœuds explorés.
1. 返回找到的目标节点以及探索的节点数。\n
2. Retourner None si l'algorithme explore tout l'arbre sans trouver le nœud objectif.
2. 如果算法探索了整棵树但没有找到目标节点，则返回 None。\n
3. Prendre en compte un budget d'exploration pour arrêter l'algorithme si aucune solution n'est trouvée après plusieurs itérations (utile pour les grands problèmes). Le budget sera également une entrée de la fonction, avec float('inf') comme valeur par défaut.
3. 考虑探索预算，如果在多次迭代后未找到解，则停止算法（对于大型问题很有用）。预算也将是函数的输入，默认值为 float('inf')。\n

Pour la frontière et l'ensemble des nœuds déjà explorés :
对于边界和已探索节点集合：\n
* La frontière doit être une liste de nœuds (objets créés par la classe Node).
* 边界必须是节点列表（由 Node 类创建的对象）。\n
* L'ensemble des nœuds explorés peut être une liste d'états.
* 已探索节点集合可以是状态列表。\n
* Pour sélectionner un élément de la frontière à explorer, utilisez la méthode *pop*. Assurez-vous de toujours prendre le dernier élément de la liste.
* 要选择要探索的边界元素，请使用 *pop* 方法。确保总是取出列表的**最后一个**元素。\n

Pour ajouter un élément à une liste, utilisez *liste.append(element)*.



### **Exercise 6**
### **练习 6**
1. Appliquez votre fonction DFS pour résoudre un Taquin 3x3 avec l'état initial [[1, 2, 3], [4, 5, 6], [0, 7, 8]]. Imprimez :
1. 应用您的 DFS 函数解决具有初始状态 [[1, 2, 3], [4, 5, 6], [0, 7, 8]] 的 3x3 推盘游戏。打印：\n
    * Le nombre de nœuds explorés par l'algorithme.
    * 算法探索的节点数。\n
    * Le chemin pour passer de l'état initial à l'état objectif (la liste des actions effectuées).
    * 从初始状态到目标状态的路径（执行的动作列表）。\n
    * Les états successifs du chemin (utilisez la fonction visualize_state).
    * 路径的连续状态（使用 visualize_state 函数）。\n
2. Répétez la même expérience pour l'état initial [[1, 2, 3], [4, 5, 0], [6, 7, 8]].
2. 对初始状态 [[1, 2, 3], [4, 5, 0], [6, 7, 8]] 重复相同的实验。\n
3. Qu'observez-vous en comparant cet algorithme avec BFS ?
3. 比较此算法与 BFS，您观察到了什么？

## **2.3 Comparaison empirique de DFS et BFS**
    ## **2.3 DFS 和 BFS 的经验比较**

Nous allons comparer la performance des deux algorithmes dans plusieurs jeux de Taquin 3x3. 
    我们将比较这两种算法在几个 3x3 推盘游戏中的性能。

### **Exercice 7**
### **练习 7**
1. Écrivez une fonction compare_algorithms qui prend en entrée :
1. 编写一个 compare_algorithms 函数，其输入为：\n
    * Une liste d’états initiaux.
    * 初始状态列表。\n
    * Un état objectif.
    * 目标状态。\n
    * Une liste d’algorithmes.
    * 算法列表。\n
2. La fonction doit résoudre chaque problème en exécutant chaque algorithme. Assurez-vous de fixer des budgets pour éviter que les algorithmes ne prennent trop de temps.
2. 该函数必须通过执行每个算法来解决每个问题。确保设置预算以防止算法花费太长时间。\n
3. Pour chaque paire (problème, algorithme), enregistrez :
3. 对于每对（问题，算法），记录：\n
    * Le nombre de nœuds explorés.
    * 探索的节点数。\n
    * La longueur du chemin trouvé entre la racine et le nœud objectif.
    * 从根节点到目标节点的路径长度。\n
    * Le temps d'exécution. Pour mesurer le temps d'exécution, vous pouvez importer time et utiliser :
    * 执行时间。要测量执行时间，您可以导入 time 并使用：\n
        * *start_time = time.time()*
        * *\# résoudre le problème*
        * *end_time = time.time()*
        * *execution_time = end_time - start_time*
4. Enregistrez les résultats dans un dictionaire.
4. 将结果保存在字典中。\n



### **Exercice 8**
### **练习 8**
Comparez les algorithmes BFS et DFS dans les instances suivantes du jeu de Taquin 3x3.
在以下 3x3 推盘游戏实例中比较 BFS 和 DFS 算法。\n
1. [[1, 2, 3], [4, 5, 0], [6, 7, 8]]
2. [[1, 2, 3], [0, 5, 6], [4, 7, 8]]    
3. [[1, 0, 3], [4, 2, 5], [7, 8, 6]]
4. [[1, 0, 3], [4, 5, 2], [7, 6, 8]]
5. [[1, 0, 3], [4, 2, 5], [6, 7, 8]]
6. [[1, 2, 3], [4, 0, 6], [7, 5, 8]]
7. [[1, 2, 3], [0, 4, 6], [7, 5, 8]]
8. [[1, 2, 3], [4, 6, 0], [7, 5, 8]]
9. [[1, 3, 6], [4, 2, 5], [7, 0, 8]]  
10. [[1, 3, 6], [4, 2, 0], [7, 5, 8]]  
11. [[1, 3, 6], [4, 0, 2], [7, 5, 8]]  
12. [[3, 1, 2], [4, 6, 5], [7, 0, 8]]  
13. [[8, 1, 2], [0, 4, 3], [7, 6, 5]]
14. [[1, 4, 2], [7, 0, 6], [5, 3, 8]]
15. [[2, 8, 3], [1, 6, 4], [7, 0, 5]]

#### Utilisez le code ci-dessous pour visualiser vos résultats

In [3]:
import pandas as pd
import numpy as np
from tabulate import tabulate ## La librairie Tabulate doit être installée. Si vous ne l'avez pas, vous pouvez simplement utiliser "print(df)" pour visualiser le tableau.

Data = []
for i in range(len(initial_states)):
    Data.append([int(i+1),results_BFS[i]['num_explorations'], results_DFS[i]['num_explorations'],results_BFS[i]['path_length'],results_DFS[i]['path_length'],
                 round(results_BFS[i]['execution_time'],2),round(results_DFS[i]['execution_time'],2)])

df = pd.DataFrame(Data, columns=["State", "Nodes explored BFS", "Nodes explored DFS", "Path length BFS", "Path length DFS", "Execution Time BFS", "Execution Time DFS"])
print(tabulate(df, headers='keys', tablefmt='pretty'))

# **Extra. Tous les jeux de taquin ont une solution ?**

En cours, nous avons dit que l'espace d'état a une taille de 9!. En réalité, seulement la moitié des configurations sont résolvables. Regardez ce [**lien**](https://fr.wikipedia.org/wiki/Taquin#Configurations_solubles_et_insolubles) et la fonction suivante pour plus d'*insights*.

In [4]:
import random

def is_solvable(puzzle):
    """Check if a 3x3 Taquin puzzle is solvable."""
    flattened = [tile for row in puzzle for tile in row if tile != 0]
    inversions = sum(
        1 for i in range(len(flattened)) for j in range(i + 1, len(flattened)) if flattened[i] > flattened[j]
    )
    return inversions % 2 == 0